# 1. Introduction (Week1)

### Topic

For this Capstone Project - The Battle of Neighborhoods, I am planning to compre the similarity and dissmilarity of two cities in US, from various aspects-such as cost of living, amenity, etc.


### Personal Background

I am working as data analyst at Dallas, Texas. Due to some family reasons, I am planning move to Boston, Massachusetts in the near future (The first time live in another state). Currently, I am actively seeking jobs and interviewing with some companyies. 

### Why this topic

Texas is such a wonderful place for living: No State income Tax and low cost of living. However, Boston is such a pricy city: State income Tax and high cost of living. I am hoping my project could provide some basic guildline on how to properly evaluate Salary Matching when they have a new opportunity.

### Dataset would be used

    1. Foursquare data of Dallas with specific address
    2. Foursquare data of Boston with specific address
    3. Cost of living (Rent/House price, Public transportation cost, food price, etc.)
    
These Foursquare data will be used for compare the living similariy (standard of living = level of happieness). Cost of living will be used to evalute the monthly cost of living in both address (which can be used to find the leverage of the salray matching ratio).


In [19]:

import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [13]:
# Shenton Way, District 01, Singapore
address = '5349 Amesbury Dr, Dallas'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dallas are {}, {}.'.format(latitude, longitude))

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Dallas are 32.85271945, -96.7655228028466.


In [14]:
neighborhood_latitude=latitude
neighborhood_longitude=longitude

CLIENT_ID = 'JMVDMC0MO0U3MKR5NL3SU3GFHT1TOLO4FYQE52DUHCCISMRW' # your Foursquare ID
CLIENT_SECRET = 'SBACRDKIXOBOXSYMYZIIKHKIJLXAR1LX2DAK3BX12KTNJQNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Central Market,Grocery Store,32.849531,-96.768632
1,Royal Thai,Thai Restaurant,32.853303,-96.768326
2,Pho is for Lovers,Vietnamese Restaurant,32.853564,-96.769565
3,World Market,Furniture / Home Store,32.852789,-96.768885
4,Pluckers Wing Bar,Wings Joint,32.852876,-96.768281
5,LA Fitness,Gym / Fitness Center,32.851931,-96.768491
6,Raising Cane's Chicken Fingers,Fried Chicken Joint,32.848603,-96.769570
7,Fast Furious Japanese Grill,Diner,32.857367,-96.768985
8,Potbelly Sandwich Shop,Sandwich Place,32.855768,-96.768504
9,Movie Trading Company,Video Store,32.856489,-96.769259
